Stokes Solver
=======

We want to solve the following Stokes block system.
\\[
\begin{bmatrix}
 K & G \\
 G^T & C
\end{bmatrix}
\begin{bmatrix}
  u\\
  p
\end{bmatrix}
=
\begin{bmatrix}
 f\\ 
 h
\end{bmatrix}.
\\]

If we apply Gaussian elimination to the above as a 2x2 block matrix system
we can write this as:

\\[
\begin{bmatrix}
  K & G\\
  0 & S
\end{bmatrix}
\begin{bmatrix}
  u\\
  p
\end{bmatrix}
=
\begin{bmatrix}
 f\\ 
 \hat{h}
\end{bmatrix},
\\]

where $S=G^{T}K^{-1}G-C$ is the Schur complement and $\hat{h}=G^{T}K^{-1}f -h$.

This system is now solved first for the pressure using the Schur complement matrix, $S$. Then a backsolve for the
velocity gives the complete solution.

Note that wherever $K^{-1}$ appears, the inverse is never explicitly calculated but is achieved via
a [PETSc](http://www.mcs.anl.gov/petsc/) solve method. While solving for the pressure, there are necessarily solves using $K$ inside of the matrix $S$.
We often refer to these as 'inner' solves.

Basic usage of the Stokes solver class involves being able to easily set up the inner solves in a few different ways
(Setting up the pressure solve is more advanced).

To illustrate some basic usage let's set up a simple problem to solve.

In [1]:
import underworld as uw
import math
from underworld import function as fn

res=128
mesh = uw.mesh.FeMesh_Cartesian("Q1/DQ0", (res,res), (0.,0.), (1.,1.))
velocityField = uw.mesh.MeshVariable(mesh,2)
velocityField.data[:] = (0.,0.)

pressureField = uw.mesh.MeshVariable(mesh.subMesh,1)
pressureField.data[:] = 0.

# Freeslip bc's
IWalls = mesh.specialSets["MinI_VertexSet"] + mesh.specialSets["MaxI_VertexSet"]
JWalls = mesh.specialSets["MinJ_VertexSet"] + mesh.specialSets["MaxJ_VertexSet"]
freeslip = uw.conditions.DirichletCondition(velocityField, (IWalls, JWalls))
# We are going to make use of one of the existing analytic solutions so that we may easily
# obtain functions for a viscosity profile and forcing terms.
# Exact solution solCx with defaults
sol = fn.analytic.SolCx()
stokesSystem = uw.systems.Stokes(velocityField,pressureField,sol.fn_viscosity,sol.fn_bodyforce,conditions=freeslip)


Now we create a solver.
The Solver class will automatically return a Stokes solver given a Stokes system.

In [2]:
solver=uw.systems.Solver(stokesSystem)

The Stokes solver will use multigrid as a preconditioner along with PETSc's
'fgmres' Krylov method by default for the 'inner' solve.

Let's run the solver now.

In [3]:
solver.solve()

Now let's set up the inner solve to do a direct solve (this will not work in parallel).

In [4]:
solver.set_inner_method("lu")

In [5]:
solver.solve()

Let's run underworld's help function on the solver.configure function.

In [6]:
help(solver.set_inner_method)

Help on method set_inner_method in module underworld.systems._bsscr:

set_inner_method(self, solve_type='mg') method of underworld.systems._bsscr.StokesSolver instance
    Configure velocity/inner solver (A11 PETSc prefix).
    
    solve_type can be one of:
    
    mg          : Geometric multigrid (default).
    mumps       : MUMPS parallel direct solver.
    superludist : SuperLU parallel direct solver.
    superlu     : SuperLU direct solver (serial only).
    lu          : LU direct solver (serial only).



We can see all the of the options that are configured in the solver using the `list()` functions for each component of the solver. These are the most useful ones.

In [7]:
solver.options.main.list()  # System level options
print( "---" )
solver.options.scr.list()   # Specifics for the schur complement solve
print( "---" )
solver.options.A11.list()   # Specifics for the inner (velocity) solve
print( "---" )
solver.options.mg.list()    # Options relevant to multigrid (if chosen)


('remove_constant_pressure_null_space', False)
('ksp_k2_type', 'NULL')
('change_backsolve', False)
('penalty', 0.0)
('pc_type', 'none')
('force_correction', True)
('k_scale_only', True)
('Q22_pc_type', 'uw')
('change_A11rhspresolve', False)
('ksp_type', 'bsscr')
('rescale_equations', False)
('restore_K', True)
---
('ksp_type', 'fgmres')
('ksp_rtol', 1e-05)
---
('pc_type', 'lu')
('_mg_active', False)
('ksp_type', 'preonly')
---
('active', False)
('levels', 8)


To learn more about the possible options, you can first take a look at 

```
uw.help(solver.options.A11) # for example
```

The options are cryptic because they follow the conventions established by `PETSc` which we use for the linear algebra and we allow you to pass any valid option through to that system. To understand this better, you will have to look at the [Petsc documentation](www.mcs.anl.gov/petsc/documentation/). To begin with, the examples in this notebook should be ok to get going. 


In [8]:
help(solver.options.A11)

Help on Options in module underworld.systems._options object:

class Options(__builtin__.object)
 |  Set PETSc options on this to pass along to PETSc KSPs
 |  
 |  ksp_type = <fgmres>    : Krylov method
 |  ksp_rtol = <1e-05>     : Relative decrease in residual norm
 |  pc_type  = <sor>       : Preconditioner type
 |  ksp_view = 'ascii'     : Print the ksp data structure at the end of the system solution
 |  ksp_converged_reason = 'ascii' : Print reason for converged or diverged solve
 |  ksp_monitor = <stdout> : Monitor preconditioned residual norm
 |  
 |  for further options see PETSc manual or set help on "options.main"
 |  
 |  Methods defined here:
 |  
 |  __init__(self)
 |  
 |  help(self)
 |  
 |  list(self)
 |      List options.
 |  
 |  reset(self)
 |      Reset values to initial defaults.
 |  
 |  set_lu(self)
 |      Set up options for LU serial solve.
 |  
 |  set_mumps(self, pc_type='lu')
 |      Set up options for MUMPS parallel solve.
 |      pc_type = "lu" or "cholesk

A useful trick is to be able to imitate the classic _"penalty method"_ which can be very efficient with modest-sized (2D) problems.  

In the penalty method, we add a term to the weak form of the Stokes equation which penalises $\lambda | \nabla \cdot \mathbf{u}| > 0$ and where $\lambda$ is a sufficiently large constant to enforce the constraint. Typically $10^7$ is considered sufficient. 

The problem with this method is that the condition number of the system is severely compromised by adding the penalty term and standard iterative methods do not work well. 

Our solvers have been configured with the penalty term and, for sufficiently robust choices of the inner solver, this can help solve problems faster (by improving pressure convergence). 

An indestructible solver like `lu` or `mumps` (Mumps is a direct solve that will work in parallel) can use very large penalties. Hence we can recreate the penalty method as follows (though it still follows the pattern of the Schur complement solver, while the classical method takes some shortcuts).

In [9]:
solver.set_inner_method("mumps")
solver.options.scr.ksp_type="cg"
solver.set_penalty(1.0e7)
solver.solve()
solver.print_stats()


 
Pressure iterations:   2
Velocity iterations:   1 (presolve)      
Velocity iterations:   2 (pressure solve)
Velocity iterations:   1 (backsolve)     
Velocity iterations:   4 (total solve)   
 
SCR RHS  solve time: 3.2747e-01
Pressure solve time: 2.9771e-02
Velocity solve time: 1.6194e-01 (backsolve)
Total solve time   : 5.9701e-01
 
Velocity solution min/max: 0.0000e+00/0.0000e+00
Pressure solution min/max: 0.0000e+00/0.0000e+00
 



Now let's go back to using multigrid. We can use a penalty here too, but the gigantic numbers won't work.

In [10]:
solver.set_inner_method("mg")
solver.set_penalty(1.0)
solver.solve()
solver.print_stats()


 
Pressure iterations:   6
Velocity iterations:   6 (presolve)      
Velocity iterations:  30 (pressure solve)
Velocity iterations:   5 (backsolve)     
Velocity iterations:  41 (total solve)   
 
SCR RHS  solve time: 9.8037e-02
Pressure solve time: 2.1733e-01
Velocity solve time: 7.0973e-02 (backsolve)
Total solve time   : 4.8103e-01
 
Velocity solution min/max: 0.0000e+00/0.0000e+00
Pressure solution min/max: 0.0000e+00/0.0000e+00
 



Let's run help on the solver itself. The whole thing is reasonably well documented and help works for all the subcomponents.

In [11]:
help(solver)

Help on StokesSolver in module underworld.systems._bsscr object:

class StokesSolver(underworld._stgermain.StgCompoundComponent)
 |  The Block Stokes Schur Complement Solver:
 |  This solves the saddle-point system
 |  
 |  [ K  G][u] = [f]
 |  [ G' C][p]   [h]
 |  
 |  via a Schur complement method.
 |  
 |  We first solve:
 |    a: S*p= G'*Ki*f - h,
 |  
 |  where S = G'*Ki*G-C and Ki = inverse of K.
 |  
 |  Then we backsolve for the velocity
 |    b: K*u = f - G*p.
 |  
 |  The effect of the inverse of K in (a) is obtained via a KSPSolve in PETSc.
 |  This has the prefix 'A11' (often called the 'inner' solve)
 |  
 |  The solve in (a) for the pressure has prefix 'scr'.
 |  
 |  Assuming the returned solver is called 'solver':
 |  
 |  It is possible to configure these solves individually via the
 |    solver.options.A11
 |  and
 |    solver.options.scr
 |  dictionaries.
 |  
 |  Try uw.help(solver.options.A11) for some details.
 |  
 |  Common configurations are provided via the
 |